# Memory Agent（记忆型 Agent）

## 回顾

我们创建了一个 chatbot，它可以将语义记忆（semantic memories）保存到单个[用户档案（user profile）](https://langchain-ai.github.io/langgraph/concepts/memory/#profile)或[集合（collection）](https://langchain-ai.github.io/langgraph/concepts/memory/#collection)中。

我们介绍了 [Trustcall](https://github.com/hinthornw/trustcall) 作为更新这两种 schema 的方法。

## 目标

现在，我们将整合之前学习的知识点，构建一个具有长期记忆能力的 [agent](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/)。

我们的 agent 名为 `task_mAIstro`，它将帮助我们管理待办事项列表（ToDo list）！

我们之前构建的 chatbot *总是*会反思对话内容并保存记忆。

而 `task_mAIstro` 会*决定何时*保存记忆（将任务项添加到待办列表）。

之前的 chatbot 总是只保存一种类型的记忆：要么是 profile，要么是 collection。

`task_mAIstro` 可以自主决定保存到用户档案（user profile）还是待办事项集合（collection of ToDo items）。

除了语义记忆（semantic memory），`task_mAIstro` 还会管理程序性记忆（procedural memory）。

这使得用户可以更新他们创建待办事项的偏好设置。

In [ ]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph trustcall langchain_core

In [ ]:
import os, getpass

def _set_env(var: str):
    # 检查环境变量是否已在操作系统中设置
    env_value = os.environ.get(var)
    if not env_value:
        # 如果未设置,提示用户输入
        env_value = getpass.getpass(f"{var}: ")
    
    # 为当前进程设置环境变量
    os.environ[var] = env_value

# 设置 LangSmith API 密钥用于追踪和监控
_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

In [ ]:
# 设置 OpenAI API 密钥
_set_env("OPENAI_API_KEY")

## 深入了解 Trustcall 的更新机制

Trustcall 用于创建和更新 JSON schemas。

如果我们想要了解 Trustcall *具体做了哪些更改*该怎么办？

例如，我们之前看到 Trustcall 有一些自己的工具来：

* 从验证失败中自我纠正 -- [查看追踪示例](https://smith.langchain.com/public/5cd23009-3e05-4b00-99f0-c66ee3edd06e/r/9684db76-2003-443b-9aa2-9a9dbc5498b7) 
* 更新现有文档 -- [查看追踪示例](https://smith.langchain.com/public/f45bdaf0-6963-4c19-8ec9-f4b7fe0f68ad/r/760f90e1-a5dc-48f1-8c34-79d6a3414ac3)

了解这些工具的运作方式对我们即将构建的 agent 非常有用。

下面我们将展示如何实现这一点！

In [ ]:
from pydantic import BaseModel, Field

# 定义记忆数据模型
class Memory(BaseModel):
    # 记忆的主要内容,例如:用户表示对学习法语感兴趣
    content: str = Field(description="The main content of the memory. For example: User expressed interest in learning about French.")

# 定义记忆集合模型
class MemoryCollection(BaseModel):
    # 关于用户的记忆列表
    memories: list[Memory] = Field(description="A list of memories about the user.")

我们可以为 Trustcall 提取器添加一个 [listener（监听器）](https://python.langchain.com/docs/how_to/lcel_cheatsheet/#add-lifecycle-listeners)。

这会将提取器执行过程中的运行信息传递给我们将要定义的 `Spy` 类。

我们的 `Spy` 类将提取关于 Trustcall 调用了哪些工具的信息。

In [ ]:
from trustcall import create_extractor
from langchain_openai import ChatOpenAI

# ============================================================
# Spy 类：监控 Trustcall 的工具调用
# ============================================================
# Python 知识点：
# - __init__: 构造函数，在创建对象时自动调用
# - __call__: 使对象可调用（callable），允许像函数一样使用对象实例
# ============================================================
class Spy:
    """
    Spy 类充当监听器（listener），用于捕获 Trustcall 执行过程中的所有工具调用。
    
    LangGraph 知识点：
    - listener 是 LCEL（LangChain Expression Language）的一个特性
    - 可以在 Runnable 的生命周期事件（如 on_start, on_end）上附加回调函数
    - 这样可以监控执行过程，而不改变执行逻辑
    """
    def __init__(self):
        """
        初始化 Spy 对象
        
        Python 知识点：
        - self 是实例本身的引用，类似其他语言的 this
        - 初始化一个空列表用于存储捕获的工具调用
        """
        # 存储所有被调用的工具信息
        self.called_tools = []

    def __call__(self, run):
        """
        使 Spy 实例可以像函数一样被调用
        
        参数:
            run: LangSmith 的运行对象，包含执行过程的详细信息
        
        Python 知识点：
        - __call__ 方法让对象实例可以被调用：spy_instance(arg) 等价于 spy_instance.__call__(arg)
        - 这是 Python 的"魔术方法"（magic method）之一
        
        算法说明：
        - 使用广度优先搜索（BFS）遍历运行树
        - 队列（queue）用于逐层遍历所有子运行
        """
        # 收集提取器调用的工具的信息
        # 使用广度优先搜索（BFS）遍历所有子运行
        q = [run]  # 初始化队列，包含根运行
        
        while q:
            r = q.pop()  # 从队列中取出一个运行对象（Python 列表的 pop() 默认移除最后一个元素）
            
            # 如果有子运行，将它们加入队列继续遍历
            if r.child_runs:
                q.extend(r.child_runs)  # extend() 将列表中的所有元素添加到队列
            
            # LangSmith 知识点：
            # - run_type 标识运行的类型（如 "chat_model", "tool", "chain" 等）
            # - 只有 chat_model 类型的运行才包含工具调用信息
            if r.run_type == "chat_model":
                # 从运行输出中提取工具调用信息
                # 导航路径：outputs -> generations -> message -> kwargs -> tool_calls
                # Python 知识点：
                # - 使用链式索引访问嵌套字典和列表
                # - [0][0] 表示取第一个 generation 的第一个消息
                self.called_tools.append(
                    r.outputs["generations"][0][0]["message"]["kwargs"]["tool_calls"]
                )

# ============================================================
# 创建 Trustcall 提取器
# ============================================================

# 初始化间谍对象，用于监控 Trustcall 的工具调用
spy = Spy()

# 初始化 OpenAI 模型
# LangChain 知识点：
# - ChatOpenAI 是 LangChain 对 OpenAI API 的封装
# - temperature=0 表示确定性输出（无随机性），适合需要一致结果的任务
model = ChatOpenAI(model="gpt-4o", temperature=0)

# 创建 Trustcall 提取器
# Trustcall 知识点：
# - Trustcall 是一个用于从对话中提取结构化信息的库
# - 它使用 LLM 来识别和提取符合特定 schema 的信息
# - 可以自动处理 schema 验证和错误纠正
trustcall_extractor = create_extractor(
    model,                    # 使用的语言模型
    tools=[Memory],           # 指定使用的工具/schema（Memory 在 cell-5 中定义）
    tool_choice="Memory",     # 强制模型使用 Memory 工具（确保总是提取记忆）
    enable_inserts=True,      # 允许插入新记忆（不仅仅是更新现有记忆）
)

# 将间谍添加为监听器
# LCEL 知识点：
# - with_listeners() 是 Runnable 的方法，用于附加生命周期监听器
# - on_end 表示在执行结束时调用监听器
# - 这样我们就能捕获 Trustcall 内部的所有工具调用，包括自我纠正和文档更新
trustcall_extractor_see_all_tool_calls = trustcall_extractor.with_listeners(on_end=spy)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# 给提取器的指令:从对话中提取记忆
instruction = """Extract memories from the following conversation:"""

# 模拟一段对话
# 包含用户介绍自己的名字和活动
conversation = [HumanMessage(content="Hi, I'm Lance."), 
                AIMessage(content="Nice to meet you, Lance."), 
                HumanMessage(content="This morning I had a nice bike ride in San Francisco.")]

# 调用提取器来提取记忆
# 将系统指令和对话历史组合在一起
result = trustcall_extractor.invoke({"messages": [SystemMessage(content=instruction)] + conversation})

In [ ]:
# 打印消息中包含的工具调用信息
# 这显示了 AI 决定调用哪些工具以及传递了什么参数
for m in result["messages"]:
    m.pretty_print()

In [ ]:
# 打印符合模式的响应(提取的记忆)
# responses 包含了按照 Memory 模式解析后的结构化数据
for m in result["responses"]: 
    print(m)

In [ ]:
# 打印元数据,包含工具调用的 ID
# 这些 ID 用于追踪和关联工具调用
for m in result["response_metadata"]: 
    print(m)

In [ ]:
# 更新对话 - 添加新的交互内容
updated_conversation = [AIMessage(content="That's great, did you do after?"), 
                        HumanMessage(content="I went to Tartine and ate a croissant."),                        
                        AIMessage(content="What else is on your mind?"),
                        HumanMessage(content="I was thinking about my Japan, and going back this winter!"),]

# 更新指令 - 现在要求更新现有记忆并创建新记忆
system_msg = """Update existing memories and create new ones based on the following conversation:"""

# 保存现有记忆,给它们分配 ID、键(工具名称)和值
# 这是为了让 Trustcall 知道哪些记忆已经存在,可以更新而不是重复创建
tool_name = "Memory"
existing_memories = [(str(i), tool_name, memory.model_dump()) for i, memory in enumerate(result["responses"])] if result["responses"] else None
existing_memories

In [ ]:
# 使用更新后的对话和现有记忆调用提取器
# Trustcall 会智能地决定是更新现有记忆还是创建新记忆
result = trustcall_extractor_see_all_tool_calls.invoke({"messages": updated_conversation, 
                                                        "existing": existing_memories})

In [ ]:
# 查看元数据中的工具调用信息
# 注意 'json_doc_id' 字段 - 它标识了被更新的现有记忆
for m in result["response_metadata"]: 
    print(m)

In [ ]:
# 查看所有工具调用的详细信息
# 可以看到 Trustcall 同时更新了现有记忆并创建了新记忆
for m in result["messages"]:
    m.pretty_print()

In [ ]:
# 查看解析后的响应
# 第一个是更新后的记忆,第二个是新创建的记忆
for m in result["responses"]:
    print(m)

In [ ]:
# 检查间谍捕获的工具调用
# 这展示了 Trustcall 内部使用的工具,包括 PatchDoc(用于更新)和 Memory(用于创建)
spy.called_tools

In [ ]:
# ============================================================
# extract_tool_info 函数：解析和格式化 Trustcall 的工具调用
# ============================================================
def extract_tool_info(tool_calls, schema_name="Memory"):
    """
    从工具调用中提取信息，包括补丁（更新）和新记忆。
    
    该函数解析 Trustcall 在执行过程中调用的工具，并将其转换为易读的格式。
    这对于理解 Trustcall 的行为非常重要，特别是区分"更新现有记忆"和"创建新记忆"。
    
    Args:
        tool_calls: 来自模型的工具调用列表（通常从 spy.called_tools 获取）
                   这是一个嵌套列表结构：[[{tool_call1}, {tool_call2}], [{tool_call3}], ...]
        schema_name: 模式工具的名称（例如 "Memory", "ToDo", "Profile"）
                    用于识别创建新记忆的工具调用
    
    Returns:
        str: 格式化的字符串，描述所有的更改操作
    
    Python 知识点：
    - 函数参数可以有默认值（schema_name="Memory"）
    - 嵌套列表遍历：for call_group in tool_calls: for call in call_group
    - 字典访问：call['name'], call['args']
    
    Trustcall 知识点：
    - PatchDoc: Trustcall 的内置工具，用于更新现有文档
    - schema_name 工具: 用于创建新记忆的工具（如 Memory, ToDo 等）
    - Trustcall 会自动决定何时使用 PatchDoc（更新）vs 创建新记忆
    """

    # 初始化变更列表，用于存储所有的更改操作
    changes = []
    
    # 遍历所有工具调用组
    # Python 知识点：双层循环遍历嵌套列表
    for call_group in tool_calls:
        for call in call_group:
            # 情况1：PatchDoc 工具调用 - 表示更新现有文档
            if call['name'] == 'PatchDoc':
                # Trustcall 知识点：
                # - json_doc_id: 被更新文档的唯一标识符
                # - planned_edits: Trustcall 计划的编辑说明（自然语言描述）
                # - patches: 实际的补丁操作（JSON Patch 格式）
                # - patches[0]['value']: 新添加或修改的值
                changes.append({
                    'type': 'update',                                # 操作类型：更新
                    'doc_id': call['args']['json_doc_id'],           # 文档 ID
                    'planned_edits': call['args']['planned_edits'],  # 编辑计划
                    'value': call['args']['patches'][0]['value']     # 新值
                })
            # 情况2：schema_name 工具调用 - 表示创建新记忆
            elif call['name'] == schema_name:
                # Python 知识点：
                # - call['args'] 包含创建新记忆所需的所有参数
                # - 这些参数符合在 Pydantic 模型中定义的 schema
                changes.append({
                    'type': 'new',              # 操作类型：新建
                    'value': call['args']       # 新记忆的完整内容
                })

    # 将结果格式化为单个字符串，方便阅读和展示
    # Python 知识点：
    # - 列表推导式（list comprehension）可以用于构建字符串列表
    # - str.join() 将列表中的字符串连接成单个字符串
    # - f-string (f"...") 用于字符串插值
    result_parts = []
    for change in changes:
        if change['type'] == 'update':
            # 格式化更新操作的输出
            result_parts.append(
                f"Document {change['doc_id']} updated:\n"      # 文档 ID
                f"Plan: {change['planned_edits']}\n"           # 更新计划
                f"Added content: {change['value']}"            # 新增内容
            )
        else:
            # 格式化新建操作的输出
            result_parts.append(
                f"New {schema_name} created:\n"                # 新建的类型
                f"Content: {change['value']}"                  # 完整内容
            )
    
    # Python 知识点：
    # - "\n\n".join() 在每个部分之间添加两个换行符，使输出更易读
    return "\n\n".join(result_parts)

# ============================================================
# 实际使用示例
# ============================================================
# 检查 spy.called_tools 以准确了解提取过程中发生了什么
schema_name = "Memory"
# 调用函数解析工具调用
# Python 知识点：
# - 将函数返回值赋给变量
# - print() 输出到标准输出
changes = extract_tool_info(spy.called_tools, schema_name)
print(changes)

## 创建 Agent

有许多不同的 [agent 架构](https://langchain-ai.github.io/langgraph/concepts/high_level/)可供选择。

这里我们将实现一个简单的 [ReAct](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/#react-implementation) agent。

这个 agent 将成为创建和管理待办事项列表的有用助手。

该 agent 可以决定更新三种类型的长期记忆：

(a) 创建或更新用户 `profile`（包含用户的一般信息）

(b) 在待办事项列表 `collection` 中添加或更新项目

(c) 更新其自身关于如何向待办事项列表添加项目的 `instructions`（指令）

In [ ]:
from typing import TypedDict, Literal

# 定义更新记忆的工具
# 这个工具让 AI 决定要更新哪种类型的记忆
class UpdateMemory(TypedDict):
    """ Decision on what memory type to update """
    # 更新类型可以是:
    # - 'user': 用户个人资料
    # - 'todo': 待办事项列表
    # - 'instructions': 如何创建待办事项的指令
    update_type: Literal['user', 'todo', 'instructions']

In [ ]:
# 再次设置 OpenAI API 密钥(如果需要)
_set_env("OPENAI_API_KEY")

## Graph 定义

我们添加一个简单的路由器 `route_message`，它做出是否保存记忆的二元决策。

记忆集合的更新由 `write_memory` 节点中的 `Trustcall` 处理，与之前一样！

In [ ]:
# ============================================================
# Memory Agent 完整实现
# ============================================================
# 这是一个完整的 LangGraph Agent 实现，展示了如何构建具有长期记忆的 AI Agent
# 
# 核心概念：
# 1. 三种记忆类型：用户资料（Profile）、待办事项（ToDo）、指令（Instructions）
# 2. ReAct 模式：推理（Reason）- 行动（Act）循环
# 3. 条件路由：根据 Agent 的决策动态选择下一步
# 4. 状态管理：短期记忆（checkpointer）+ 长期记忆（store）
# ============================================================

import uuid
from IPython.display import Image, display

from datetime import datetime
from trustcall import create_extractor
from typing import Optional
from pydantic import BaseModel, Field

from langchain_core.runnables import RunnableConfig
from langchain_core.messages import merge_message_runs, HumanMessage, SystemMessage

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, MessagesState, END, START
from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore

from langchain_openai import ChatOpenAI

# 初始化模型
# LangChain 知识点：
# - temperature=0 确保输出的确定性，适合需要一致行为的 Agent
model = ChatOpenAI(model="gpt-4o", temperature=0)

# ============================================================
# 数据模型定义 - 使用 Pydantic 定义结构化 Schema
# ============================================================
# Pydantic 知识点：
# - BaseModel: Pydantic 的基类，提供数据验证和序列化功能
# - Field: 为字段添加元数据（描述、默认值、验证规则等）
# - Optional: 表示字段可以为 None
# - 这些模型会被转换为 LLM 可以理解的 JSON Schema

class Profile(BaseModel):
    """
    用户资料模式 - 存储用户的基本信息
    
    这个模式定义了我们想要记住的关于用户的信息类型。
    Trustcall 会使用这个模式来指导 LLM 提取相关信息。
    """
    name: Optional[str] = Field(
        description="用户的姓名",
        default=None  # None 表示字段可选，未提供时不会报错
    )
    location: Optional[str] = Field(
        description="用户的位置/居住地",
        default=None
    )
    job: Optional[str] = Field(
        description="用户的职业",
        default=None
    )
    connections: list[str] = Field(
        description="用户的个人关系，如家人、朋友或同事",
        default_factory=list  # default_factory 用于可变默认值（如列表）
    )
    interests: list[str] = Field(
        description="用户的兴趣爱好", 
        default_factory=list
    )

class ToDo(BaseModel):
    """
    待办事项模式 - 定义待办任务的结构
    
    这个详细的模式确保每个待办事项都包含足够的信息来有效管理。
    """
    task: str = Field(
        description="需要完成的任务描述"
    )
    time_to_complete: Optional[int] = Field(
        description="完成任务的预估时间（分钟）"
    )
    deadline: Optional[datetime] = Field(
        description="任务的截止日期时间（如适用）",
        default=None
    )
    solutions: list[str] = Field(
        description="具体的、可执行的解决方案列表（如具体想法、服务提供商或完成任务的具体选项）",
        min_items=1,  # Pydantic 验证：至少需要一个解决方案
        default_factory=list
    )
    status: Literal["not started", "in progress", "done", "archived"] = Field(
        description="任务的当前状态",
        default="not started"  # Literal 限制字段只能是指定的几个值之一
    )

# ============================================================
# 创建用于更新用户资料的 Trustcall 提取器
# ============================================================
# Trustcall 知识点：
# - 为不同的记忆类型创建不同的提取器
# - Profile 提取器专门用于提取和更新用户信息
profile_extractor = create_extractor(
    model,
    tools=[Profile],           # 只使用 Profile 工具
    tool_choice="Profile",     # 强制使用 Profile
)

# ============================================================
# 系统提示模板 - Agent 的"大脑"
# ============================================================
# 提示工程知识点：
# - 使用明确的结构（编号列表、XML 标签）帮助 LLM 理解
# - 提供当前上下文（profile, todo, instructions）
# - 明确决策规则和行为准则

MODEL_SYSTEM_MESSAGE = """You are a helpful chatbot. 

You are designed to be a companion to a user, helping them keep track of their ToDo list.

You have a long term memory which keeps track of three things:
1. The user's profile (general information about them) 
2. The user's ToDo list
3. General instructions for updating the ToDo list

Here is the current User Profile (may be empty if no information has been collected yet):
<user_profile>
{user_profile}
</user_profile>

Here is the current ToDo List (may be empty if no tasks have been added yet):
<todo>
{todo}
</todo>

Here are the current user-specified preferences for updating the ToDo list (may be empty if no preferences have been specified yet):
<instructions>
{instructions}
</instructions>

Here are your instructions for reasoning about the user's messages:

1. Reason carefully about the user's messages as presented below. 

2. Decide whether any of the your long-term memory should be updated:
- If personal information was provided about the user, update the user's profile by calling UpdateMemory tool with type `user`
- If tasks are mentioned, update the ToDo list by calling UpdateMemory tool with type `todo`
- If the user has specified preferences for how to update the ToDo list, update the instructions by calling UpdateMemory tool with type `instructions`

3. Tell the user that you have updated your memory, if appropriate:
- Do not tell the user you have updated the user's profile
- Tell the user them when you update the todo list
- Do not tell the user that you have updated instructions

4. Err on the side of updating the todo list. No need to ask for explicit permission.

5. Respond naturally to user user after a tool call was made to save memories, or if no tool call was made."""

# Trustcall 的指令 - 告诉它如何反思对话
TRUSTCALL_INSTRUCTION = """Reflect on following interaction. 

Use the provided tools to retain any necessary memories about the user. 

Use parallel tool calling to handle updates and insertions simultaneously.

System Time: {time}"""

# 更新待办事项指令的提示
CREATE_INSTRUCTIONS = """Reflect on the following interaction.

Based on this interaction, update your instructions for how to update ToDo list items. 

Use any feedback from the user to update how they like to have items added, etc.

Your current instructions are:

<current_instructions>
{current_instructions}
</current_instructions>"""

# ============================================================
# 节点函数定义 - Graph 的核心逻辑
# ============================================================
# LangGraph 知识点：
# - 节点是有状态的函数，接收 state, config, store 作为参数
# - 返回的字典会合并到状态中（MessagesState 支持消息列表的自动追加）
# - 节点可以访问 store（长期记忆）和 state（当前会话状态）

def task_mAIstro(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """
    主 Agent 节点 - 负责与用户交互并决定是否更新记忆
    
    这是 Agent 的"大脑"，执行以下操作：
    1. 从 store 加载长期记忆（跨会话持久化）
    2. 使用记忆个性化系统提示
    3. 调用 LLM 生成响应
    4. 可能调用 UpdateMemory 工具来触发记忆更新
    
    参数:
        state: MessagesState - 包含对话历史
        config: RunnableConfig - 包含配置信息（如 user_id）
        store: BaseStore - 长期记忆存储
    
    返回:
        dict: 包含新消息的字典（会被追加到 state["messages"]）
    
    LangGraph 知识点：
    - 节点函数的签名必须匹配：(state, config, store)
    - config["configurable"] 用于传递运行时配置
    - store.search() 用于检索记忆
    """
    
    # 从配置中获取用户 ID
    # 这允许为不同用户维护独立的记忆空间
    user_id = config["configurable"]["user_id"]

    # ========================================
    # 步骤 1: 检索用户资料记忆
    # ========================================
    # LangGraph Store 知识点：
    # - 使用命名空间（namespace）组织数据：(type, user_id)
    # - search() 返回匹配的记忆列表
    namespace = ("profile", user_id)
    memories = store.search(namespace)
    if memories:
        # 获取第一个（应该只有一个用户资料）
        user_profile = memories[0].value
    else:
        user_profile = None

    # ========================================
    # 步骤 2: 检索待办事项记忆
    # ========================================
    namespace = ("todo", user_id)
    memories = store.search(namespace)
    # 将所有待办事项格式化为字符串
    # Python 知识点：
    # - 列表推导式: [expression for item in iterable]
    # - str.join() 连接字符串
    todo = "\n".join(f"{mem.value}" for mem in memories)

    # ========================================
    # 步骤 3: 检索自定义指令（程序性记忆）
    # ========================================
    namespace = ("instructions", user_id)
    memories = store.search(namespace)
    if memories:
        instructions = memories[0].value
    else:
        instructions = ""
    
    # ========================================
    # 步骤 4: 使用记忆填充系统消息模板
    # ========================================
    # Python 知识点：
    # - str.format() 方法用于字符串插值
    # - 将检索到的记忆注入到提示模板中
    system_msg = MODEL_SYSTEM_MESSAGE.format(
        user_profile=user_profile, 
        todo=todo, 
        instructions=instructions
    )

    # ========================================
    # 步骤 5: 调用 LLM 生成响应
    # ========================================
    # LangChain 知识点：
    # - bind_tools() 让模型知道可以调用哪些工具
    # - parallel_tool_calls=False 确保一次只调用一个工具（避免同时更新多种记忆）
    # - 消息列表：[SystemMessage, ...历史消息]
    response = model.bind_tools(
        [UpdateMemory],              # 提供 UpdateMemory 工具
        parallel_tool_calls=False    # 顺序调用工具
    ).invoke([SystemMessage(content=system_msg)] + state["messages"])

    # 返回新消息（会自动追加到 state["messages"]）
    return {"messages": [response]}

def update_profile(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """
    更新用户资料节点 - 使用 Trustcall 提取并保存用户信息
    
    工作流程：
    1. 获取现有的用户资料（如果存在）
    2. 使用 Trustcall 分析对话历史
    3. 更新或创建用户资料
    4. 保存到 store
    5. 返回工具消息确认更新
    
    LangGraph 知识点：
    - 工具节点必须返回 ToolMessage 来响应 Agent 的工具调用
    - ToolMessage 的 tool_call_id 必须匹配 Agent 发出的工具调用 ID
    """
    
    # 从配置中获取用户 ID
    user_id = config["configurable"]["user_id"]

    # 定义记忆的命名空间
    namespace = ("profile", user_id)

    # ========================================
    # 检索现有记忆作为上下文
    # ========================================
    # Trustcall 知识点：
    # - Trustcall 需要知道现有记忆，以便决定更新还是创建新记忆
    existing_items = store.search(namespace)

    # 为 Trustcall 格式化现有记忆
    # 格式：[(key, tool_name, value), ...]
    # - key: 记忆的唯一标识符
    # - tool_name: 对应的工具名称
    # - value: 记忆的实际内容
    tool_name = "Profile"
    existing_memories = (
        [(existing_item.key, tool_name, existing_item.value)
         for existing_item in existing_items]
        if existing_items
        else None
    )

    # ========================================
    # 准备 Trustcall 的输入消息
    # ========================================
    # LangChain 知识点：
    # - merge_message_runs() 合并连续的同类型消息，减少 token 使用
    # - state["messages"][:-1] 排除最后一条消息（Agent 的工具调用）
    # - 包含当前系统时间帮助 Trustcall 理解时间相关信息
    TRUSTCALL_INSTRUCTION_FORMATTED = TRUSTCALL_INSTRUCTION.format(
        time=datetime.now().isoformat()
    )
    updated_messages = list(merge_message_runs(
        messages=[SystemMessage(content=TRUSTCALL_INSTRUCTION_FORMATTED)] + state["messages"][:-1]
    ))

    # ========================================
    # 调用 Trustcall 提取器
    # ========================================
    # Trustcall 会：
    # 1. 分析对话
    # 2. 识别用户信息
    # 3. 决定更新现有资料还是创建新资料
    # 4. 返回结构化的 Profile 对象
    result = profile_extractor.invoke({
        "messages": updated_messages, 
        "existing": existing_memories
    })

    # ========================================
    # 保存 Trustcall 的记忆到 store
    # ========================================
    # Python 知识点：
    # - zip() 同时遍历多个可迭代对象
    # - result["responses"]: 提取的 Profile 对象列表
    # - result["response_metadata"]: 包含 ID 等元数据
    for r, rmeta in zip(result["responses"], result["response_metadata"]):
        # store.put() 保存记忆
        # - namespace: 组织结构
        # - key: 记忆 ID（使用现有 ID 或生成新 UUID）
        # - value: 记忆内容（转换为 JSON）
        store.put(
            namespace,
            rmeta.get("json_doc_id", str(uuid.uuid4())),  # 获取或生成 ID
            r.model_dump(mode="json"),  # Pydantic 模型转 JSON
        )
    
    # ========================================
    # 返回工具消息
    # ========================================
    # LangGraph 知识点：
    # - 必须返回 ToolMessage 来响应 Agent 的工具调用
    # - tool_call_id 从最后一条消息中获取
    tool_calls = state['messages'][-1].tool_calls
    return {"messages": [{
        "role": "tool",                    # 工具消息
        "content": "updated profile",      # 简单确认消息
        "tool_call_id": tool_calls[0]['id']  # 匹配工具调用 ID
    }]}

def update_todos(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """
    更新待办事项节点 - 使用 Trustcall 管理待办事项列表
    
    与 update_profile 类似，但：
    1. 使用 ToDo 模式而不是 Profile
    2. 使用 Spy 监听器捕获详细的更改信息
    3. 返回详细的更改描述给用户
    
    这展示了如何为不同类型的记忆使用不同的 Trustcall 配置。
    """
    
    # 从配置中获取用户 ID
    user_id = config["configurable"]["user_id"]

    # 定义记忆的命名空间
    namespace = ("todo", user_id)

    # 检索现有待办事项
    existing_items = store.search(namespace)

    # 为 Trustcall 格式化现有记忆
    tool_name = "ToDo"
    existing_memories = (
        [(existing_item.key, tool_name, existing_item.value)
         for existing_item in existing_items]
        if existing_items
        else None
    )

    # 合并聊天历史和指令
    TRUSTCALL_INSTRUCTION_FORMATTED = TRUSTCALL_INSTRUCTION.format(
        time=datetime.now().isoformat()
    )
    updated_messages = list(merge_message_runs(
        messages=[SystemMessage(content=TRUSTCALL_INSTRUCTION_FORMATTED)] + state["messages"][:-1]
    ))

    # ========================================
    # 初始化间谍以查看 Trustcall 的工具调用
    # ========================================
    # 这让我们能看到 Trustcall 内部做了什么（创建 vs 更新）
    spy = Spy()
    
    # 创建用于更新待办事项的 Trustcall 提取器
    # 注意 enable_inserts=True，允许插入新的待办事项
    todo_extractor = create_extractor(
        model,
        tools=[ToDo],
        tool_choice=tool_name,
        enable_inserts=True  # 关键：允许创建新待办事项
    ).with_listeners(on_end=spy)  # 添加间谍监听器

    # 调用提取器
    result = todo_extractor.invoke({
        "messages": updated_messages, 
        "existing": existing_memories
    })

    # 保存 Trustcall 的记忆到存储中
    for r, rmeta in zip(result["responses"], result["response_metadata"]):
        store.put(
            namespace,
            rmeta.get("json_doc_id", str(uuid.uuid4())),
            r.model_dump(mode="json"),
        )
    
    # ========================================
    # 提取并返回详细的更改信息
    # ========================================
    # 这里我们使用之前定义的 extract_tool_info 函数
    # 来创建用户友好的更改描述
    tool_calls = state['messages'][-1].tool_calls

    # 提取 Trustcall 所做的更改
    todo_update_msg = extract_tool_info(spy.called_tools, tool_name)
    
    # 返回包含详细更改信息的工具消息
    # 这让用户能看到具体做了什么更改
    return {"messages": [{
        "role": "tool",
        "content": todo_update_msg,        # 详细的更改描述
        "tool_call_id": tool_calls[0]['id']
    }]}

def update_instructions(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """
    更新指令节点 - 管理程序性记忆
    
    程序性记忆（Procedural Memory）：
    - 不是"记住什么"，而是"记住如何做"
    - 用户可以指定他们希望 Agent 如何行为的偏好
    - 例如："创建待办事项时，总是包含本地商家"
    
    这展示了 Agent 可以"学习"用户的偏好并在未来应用。
    """
    
    # 从配置中获取用户 ID
    user_id = config["configurable"]["user_id"]
    
    namespace = ("instructions", user_id)

    # ========================================
    # 获取现有指令
    # ========================================
    # Store 知识点：
    # - get() 用于获取特定 key 的值
    # - search() 用于搜索命名空间中的所有值
    existing_memory = store.get(namespace, "user_instructions")
    
    # ========================================
    # 让模型基于对话更新指令
    # ========================================
    # 这里不使用 Trustcall，而是直接让 LLM 生成新指令
    # 因为指令是自由格式的文本，不需要结构化提取
    system_msg = CREATE_INSTRUCTIONS.format(
        current_instructions=existing_memory.value if existing_memory else None
    )
    
    # 调用模型生成新指令
    # Python 知识点：
    # - state['messages'][:-1] 排除最后一条消息（工具调用）
    # - 添加一条 HumanMessage 明确要求更新指令
    new_memory = model.invoke([
        SystemMessage(content=system_msg)
    ] + state['messages'][:-1] + [
        HumanMessage(content="Please update the instructions based on the conversation")
    ])

    # ========================================
    # 覆盖存储中的现有指令
    # ========================================
    # 注意：指令使用固定的 key "user_instructions"
    # 这样每个用户只有一组指令（会被覆盖而不是追加）
    key = "user_instructions"
    store.put(namespace, key, {"memory": new_memory.content})
    
    # 返回工具消息，确认更新完成
    tool_calls = state['messages'][-1].tool_calls
    return {"messages": [{
        "role": "tool",
        "content": "updated instructions",
        "tool_call_id": tool_calls[0]['id']
    }]}

# ============================================================
# 条件边 - 动态路由逻辑
# ============================================================
# LangGraph 知识点：
# - 条件边（conditional edge）根据函数返回值决定下一个节点
# - 返回值必须是节点名称或 END
# - Literal 类型注解帮助类型检查和文档

def route_message(state: MessagesState, config: RunnableConfig, store: BaseStore) -> Literal[END, "update_todos", "update_instructions", "update_profile"]:
    """
    路由函数 - 决定接下来执行哪个节点
    
    根据 Agent 的工具调用决定：
    - 如果没有工具调用 -> 结束对话（END）
    - 如果调用了 UpdateMemory 工具 -> 根据 update_type 路由到相应节点
    
    参数:
        state: 当前状态
        config: 配置
        store: 存储（未使用，但签名需要匹配）
    
    返回:
        str: 下一个节点的名称或 END
    
    LangGraph 知识点：
    - 条件边函数的返回值决定 Graph 的执行流程
    - 这实现了 ReAct 模式的"决策"部分
    """
    
    # 获取最后一条消息（Agent 的响应）
    message = state['messages'][-1]
    
    # 检查是否有工具调用
    if len(message.tool_calls) == 0:
        # 没有工具调用 -> Agent 认为不需要更新记忆 -> 结束
        return END
    else:
        # 有工具调用 -> 解析工具调用参数
        tool_call = message.tool_calls[0]
        
        # 根据 update_type 参数路由到相应节点
        if tool_call['args']['update_type'] == "user":
            return "update_profile"
        elif tool_call['args']['update_type'] == "todo":
            return "update_todos"
        elif tool_call['args']['update_type'] == "instructions":
            return "update_instructions"
        else:
            # 意外的 update_type -> 抛出错误
            raise ValueError(f"Unknown update_type: {tool_call['args']['update_type']}")

# ============================================================
# Graph 构建 - 组装所有部分
# ============================================================
# LangGraph 知识点：
# - StateGraph: 管理状态的计算图
# - MessagesState: 内置状态，自动管理消息列表
# - 节点：执行计算的函数
# - 边：定义执行顺序
# - 条件边：根据条件动态路由

# 创建 StateGraph
# MessagesState 提供了消息列表的自动管理
builder = StateGraph(MessagesState)

# ========================================
# 添加节点
# ========================================
# LangGraph 知识点：
# - add_node() 将函数注册为 Graph 的节点
# - 节点名称用于在边中引用
builder.add_node(task_mAIstro)           # 主 Agent 节点
builder.add_node(update_todos)           # 更新待办事项节点
builder.add_node(update_profile)         # 更新用户资料节点
builder.add_node(update_instructions)    # 更新指令节点

# ========================================
# 添加边 - 定义执行流程
# ========================================
# LangGraph 知识点：
# - add_edge(): 无条件边，总是执行
# - add_conditional_edges(): 条件边，根据函数返回值决定下一步
# - START: 特殊节点，表示 Graph 的入口
# - END: 特殊节点，表示 Graph 的出口

# 定义记忆提取流程：
# START -> task_mAIstro -> (条件路由) -> update_* -> task_mAIstro -> END
#
# 流程说明：
# 1. Graph 从 START 开始
# 2. 执行 task_mAIstro（Agent 推理和响应）
# 3. route_message 决定下一步：
#    - 如果需要更新记忆 -> 执行相应的 update_* 节点
#    - 如果不需要 -> 直接结束（END）
# 4. 如果执行了 update_* 节点，再次回到 task_mAIstro 生成最终响应
# 5. 第二次从 task_mAIstro 出来时，应该不再有工具调用，直接结束

builder.add_edge(START, "task_mAIstro")  # 入口边

# 条件边：根据 route_message 的返回值决定下一步
builder.add_conditional_edges("task_mAIstro", route_message)

# 更新节点完成后，返回到 task_mAIstro 生成最终响应
builder.add_edge("update_todos", "task_mAIstro")
builder.add_edge("update_profile", "task_mAIstro")
builder.add_edge("update_instructions", "task_mAIstro")

# ========================================
# 创建记忆管理器
# ========================================
# LangGraph 知识点：
# - InMemoryStore: 跨线程（长期）记忆，在不同会话间持久化
# - MemorySaver: 线程内（短期）记忆，保存单个会话的状态
# - 这两种记忆的结合实现了完整的记忆管理系统

# 跨线程（长期）记忆的存储
# 用于保存 Profile、ToDo、Instructions
across_thread_memory = InMemoryStore()

# 线程内（短期）记忆的检查点
# 用于保存对话历史和 Graph 状态
within_thread_memory = MemorySaver()

# ========================================
# 编译 Graph
# ========================================
# LangGraph 知识点：
# - compile() 将 Graph 转换为可执行的 Runnable
# - checkpointer: 用于保存和恢复 Graph 状态（支持中断和恢复）
# - store: 用于长期记忆存储
graph = builder.compile(
    checkpointer=within_thread_memory,  # 短期记忆（对话状态）
    store=across_thread_memory          # 长期记忆（用户数据）
)

# ========================================
# 可视化 Graph
# ========================================
# LangGraph 知识点：
# - get_graph() 获取 Graph 的表示
# - xray=1: 显示内部结构（包括隐藏的节点）
# - draw_mermaid_png(): 生成 Mermaid 格式的流程图
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

In [ ]:
# 提供线程 ID 用于短期(线程内)记忆
# 提供用户 ID 用于长期(跨线程)记忆
config = {"configurable": {"thread_id": "1", "user_id": "Lance"}}

# 用户输入以创建个人资料记忆
input_messages = [HumanMessage(content="My name is Lance. I live in SF with my wife. I have a 1 year old daughter.")]

# 运行图并流式输出结果
# stream_mode="values" 表示流式输出完整的状态值
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [ ]:
# 用户输入创建待办事项
input_messages = [HumanMessage(content="My wife asked me to book swim lessons for the baby.")]

# 运行图
# AI 会识别这是一个待办事项,调用 update_todos 节点
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [ ]:
# 用户输入以更新创建待办事项的指令
# 这是程序性记忆 - AI 学习用户的偏好
input_messages = [HumanMessage(content="When creating or updating ToDo items, include specific local businesses / vendors.")]

# 运行图
# AI 会识别这是指令更新,调用 update_instructions 节点
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [ ]:
# 检查更新后的指令
user_id = "Lance"

# 搜索指令命名空间
for memory in across_thread_memory.search(("instructions", user_id)):
    print(memory.value)

In [ ]:
# 用户输入新的待办事项
# 注意 AI 现在会应用之前学到的指令(包含本地商家)
input_messages = [HumanMessage(content="I need to fix the jammed electric Yale lock on the door.")]

# 运行图
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [ ]:
# 检查待办事项的命名空间
user_id = "Lance"

# 搜索所有待办事项
for memory in across_thread_memory.search(("todo", user_id)):
    print(memory.value)

In [ ]:
# 用户输入以更新现有待办事项
# Trustcall 会智能地识别这是对现有任务的更新,而不是新任务
input_messages = [HumanMessage(content="For the swim lessons, I need to get that done by end of November.")]

# 运行图
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

我们可以看到 Trustcall 执行了现有记忆的打补丁（patching）操作：

https://smith.langchain.com/public/4ad3a8af-3b1e-493d-b163-3111aa3d575a/r

In [ ]:
# 用户输入新的待办事项
# 注意 AI 如何在创建新任务的同时,也智能地更新了相关的现有任务
input_messages = [HumanMessage(content="Need to call back City Toyota to schedule car service.")]

# 运行图
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [ ]:
# 再次检查待办事项列表
# 可以看到所有任务,包括新创建的和更新的
user_id = "Lance"

# 搜索所有待办事项
for memory in across_thread_memory.search(("todo", user_id)):
    print(memory.value)

现在我们可以创建一个新线程（thread）。

这会创建一个新的会话。

保存到长期记忆中的 Profile、ToDos 和 Instructions 都可以被访问。

In [ ]:
# 创建一个新线程(新会话)
# 使用不同的 thread_id,但相同的 user_id
# 这样可以访问之前保存的长期记忆(资料、待办事项、指令)
config = {"configurable": {"thread_id": "2", "user_id": "Lance"}}

# 与聊天机器人交互
# AI 会根据之前保存的待办事项列表来回答
input_messages = [HumanMessage(content="I have 30 minutes, what tasks can I get done?")]

# 运行图
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [ ]:
# 继续对话
# AI 记得之前的待办事项,并提供具体的选项
input_messages = [HumanMessage(content="Yes, give me some options to call for swim lessons.")]

# 运行图
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

追踪链接（Trace）：

https://smith.langchain.com/public/84768705-be91-43e4-8a6f-f9d3cee93782/r

## Studio

![Screenshot 2024-11-04 at 1.00.19 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/6732cfb05d9709862eba4e6c_Screenshot%202024-11-11%20at%207.46.40%E2%80%AFPM.png)